In [ ]:
from typing import List, Tuple, Dict, Optional, Union
from abc import ABC, abstractmethod

import pandas as pd
from pathlib import Path
from ipywidgets.widgets.widget_box import VBox
import ipywidgets as w
from ipyfilechooser import FileChooser
from IPython.display import display
import inspect

from dcl_stats_n_plots.main_refactored import Session
from dcl_stats_n_plots.database import Configs
from dcl_stats_n_plots import stats_refactored
from dcl_stats_n_plots import plots_refactored

/home/ds/anaconda3/envs/DCLwidgets_clone/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.0, the latest is 0.5.1.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs


In [ ]:

def launch_gui():
    display(GUI().widget)

In [ ]:

class GUI:
    
    def __init__(self):
        self.uploader = w.FileUpload(accept=('.xlsx, .csv'), multiple=False, layout = {'width': '15%'})
        available_statistical_tests = self.get_available_statistical_tests()
        self.stats_selection = w.Dropdown(options = available_statistical_tests, layout = {'width': '60%'})
        self.confirm_stats_selection_button = w.Button(description = 'confirm selection and compute stats', layout = {'width': '25%'})
        user_info_0 = w.Label(value = 'Please select a file (.xlsx or .csv) you would like to upload,\
                             choose the appropriate statistical test & then confirm your selection to get started:')
        self.widget = w.VBox([user_info_0,
                              w.HBox([self.uploader, self.stats_selection, self.confirm_stats_selection_button])])
        
        self.confirm_stats_selection_button.on_click(self.build_and_change_to_tabs_ui)
        
        
    def get_available_statistical_tests(self) -> List[Tuple[stats_refactored.StatisticalTest, str]]:
        available_statistical_tests = [] # list of tuples like (description_string, stats_test_obj)
        for name, obj in inspect.getmembers(stats_refactored):
            if name.endswith('Stats'):
                available_statistical_tests.append((obj().name_displayed_in_gui, obj))
        return available_statistical_tests
    
    
    def build_and_change_to_tabs_ui(self, b) -> None:
        self.session = Session()
        self.session.upload_data_via_gui(uploader_value = self.uploader.value)
        self.session.calculate_stats(statistical_test = self.stats_selection.value)
        
        self.stats_tab = StatisticsTab(gui = self)
        self.plot_tab = PlotTab(gui = self)
        self.configurations_tab = ConfigsTab(gui = self)
        #self.io_tab = IOTab(gui = self)
        
        tabs_ui = w.Tab([self.stats_tab.widget, self.plot_tab.widget, self.configurations_tab.widget])
        tabs_ui.set_title(0, 'statistics corner')
        tabs_ui.set_title(1, 'generated plot')
        tabs_ui.set_title(2, 'customize your plot')
        
        # might require some initalization of configs?
        #self.plot_tab.update_the_plot()
        
        self.widget.children = (tabs_ui, )
        

In [ ]:
class PlainTab(ABC):
    
    def __init__(self, gui: GUI) -> None:
        self.gui = gui
        self.widget = self.create_widget()
        
    
    @abstractmethod
    def create_widget(self) -> VBox:
        pass

In [ ]:
class StatisticsTab(PlainTab):
    
    def create_widget(self) -> VBox:
        user_information_strings = self.create_user_information_strings()
        user_information_labels = []
        for user_info in user_information_strings:
            user_information_labels.append(w.Label(value = user_info))
        user_information = w.VBox(user_information_labels)
        self.display_stats_df = w.Output()
        self.export_stats = w.Button(description = 'export statistical results', layout = {'width': '25%'})
        widget = w.VBox([user_information, self.display_stats_df, self.export_stats])
        with self.display_stats_df:
            display(self.gui.session.database.stats_results['summary_stats']['full_test_results'])
        return widget
        
                                        
    def create_user_information_strings(self) -> List[str]:
        user_info_strings = []
        summary_stats = self.gui.session.database.stats_results['summary_stats'].copy()
        performed_test = summary_stats['performed_test']
        summary_p_value = summary_stats['p_value']
        summary_stars_str = summary_stats['stars_str']
        user_info_strings.append(f'I used a {performed_test} to analyze your data \
                                 and the resulting p-value is: {round(summary_p_value, 4)}.')
        if summary_p_value <= 0.05:
            user_info_strings.append('Thus, there is a *significant* difference in your data. \
                                     See the table below for more detailed information:')
        else:
            user_info_strings.append('Thus, there is *no* significant difference in your data. \
                                     Unfortunately, this also means that you have to')
            user_info_strings.append('ignore the results of pairwise comparisons that may be listed below, \
                                     even if they show p-values <= 0.05')
        return user_info_strings

In [ ]:
class PlotTab(PlainTab):
    
    def create_widget(self) -> VBox:
        available_plot_types = self.get_available_plot_types()
        self.select_plot_type = w.Dropdown(description = 'Please select a plot type: ',
                                           options = available_plot_types, 
                                           layout = {'width': '65%'},
                                           style = {'description_width': 'initial'})
        self.update_plot = w.Button(description = 'update the plot', layout = {'width': '15%'})
        self.export_plot = w.Button(description = 'export the plot', layout = {'width': '15%'})
        self.display_plot = w.Output()
        self.update_plot.on_click(self.update_the_plot)
        self.export_plot.on_click(self.export_the_plot)
        widget = w.VBox([w.HBox([self.select_plot_type, self.update_plot, self.export_plot]),
                         self.display_plot])
        return widget
    
    
    def get_available_plot_types(self) -> List[str]:
        return self.gui.stats_selection.value().plot_handler().plot_options_displayed_in_gui        
        
    
    def update_the_plot(self, b):
        self.gui.session.database.configs = self.gui.configurations_tab.update_configs()
        with self.display_plot:
            self.display_plot.clear_output()
            self.gui.session.create_plot()
            

    def export_the_plot(self, b):
        pass

In [ ]:
class ConfigsTab(PlainTab):
    
    def create_widget(self):
        user_info0 = w.Label(value = 'Please use the interactive widgets below to customize the plot to your needs.')
        user_info1 = w.Label(value = 'You also have the option to export your current settings,\
                                      in order to upload & set them again in a subsequent session.')
        user_information = w.VBox([user_info0, user_info1])
        self.export_configs = w.Button(description = 'export your current settings', layout = {'width': '25%'})
        self.upload_configs = w.FileUpload(accept=('.json'), multiple=False, layout = {'width': '15%'})
        self.set_uploaded_configs = w.Button(description = 'update to uploaded settings', layout = {'width': '25%'})
        
        stats_annotation_accordion_tab = self.initialize_stats_annotation_accordion_tab()
        axes_accordion_tab = self.initialize_axes_accordion_tab()
        additional_configs_accordion_tab = self.initialize_additional_configs_accordion_tab()
        
        accordion = w.Accordion([stats_annotation_accordion_tab, axes_accordion_tab, additional_configs_accordion_tab])
        accordion.set_title(0, 'Customize how statistics are annotated in the plot')
        accordion.set_title(1, 'Customize axes')
        accordion.set_title(2, 'Customize additional other features of the plot')
        
        widget = w.VBox([user_information,
                         w.HBox([self.export_configs, self.upload_configs, self.set_uploaded_configs]),
                         accordion])
        
        self.export_configs.on_click(self.export_current_settings)
        self.set_uploaded_configs.on_click(self.extract_and_set_uploaded_configs)

        return widget


    def initialize_stats_annotation_accordion_tab(self):
        
        return w.Accordion()


    def initialize_axes_accordion_tab(self):
        x_axis_vbox = self.initialize_x_axis_vbox()
        y_axis_vbox = self.initialize_y_axis_vbox()
        common_axis_features_vbox = self.initialize_common_axis_features_vbox()
        accordion = w.Accordion([x_axis_vbox, y_axis_vbox, common_axis_features_vbox])
        return w.VBox([accordion])

    
    def initialize_x_axis_vbox(self) -> VBox:
        initial_xlabel_order = ''
        for group_id in self.gui.session.database.stats_results['df_infos']['all_group_ids']:
            if group_id != self.gui.session.database.stats_results['df_infos']['all_group_ids'][-1]:
                initial_xlabel_order += f'{group_id}, '
            else:
                initial_xlabel_order += f'{group_id}'
        self.l_xlabel_order = w.Text(value = initial_xlabel_order,
                                     placeholder = 'Specify the desired order of the x-axis \
                                     labels with individual labels separated by a comma',
                                     description = 'x-axis label order (separated by comma):',
                                     layout = {'width': '90%'},
                                     style = {'description_width': 'initial'})
        self.xaxis_label_text = w.Text(value = 'group_IDs', placeholder = 'group_IDs', 
                                       description = 'x-axis title:', layout = {'width': '36%'})
        self.xaxis_label_fontsize = w.IntSlider(value = 12, min = 8, max = 40, 
                                                step = 1, description = 'fontsize:', layout = {'width': '28%'})
        self.xaxis_label_color = w.ColorPicker(concise = False, description = 'font color', 
                                               value = '#000000', layout = {'width': '28%'})
        #needs to be implemented, also with corresponding exception to properly update configs
        initial_hue_order = ''
        self.hue_order = w.Text(value = initial_hue_order,
                                 placeholder = 'Specify the desired group order with individual groups separated by a comma',
                                 description = 'group order (separated by comma):',
                                 layout = {'width': '90%', 'visibility': 'hidden'},
                                 style = {'description_width': 'initial'})
        row_0 = w.HBox([self.xaxis_label_text, self.xaxis_label_fontsize, self.xaxis_label_color])
        row_1 = self.l_xlabel_order
        row_2 = self.hue_order
        return w.VBox([row_0, row_1, row_2])
    
    
    def initialize_y_axis_vbox(self) -> VBox:
        self.yaxis_label_text = w.Text(value = 'data', placeholder = 'data', description = 'y-axis title:', 
                                       layout = {'width': '36%'})
        self.yaxis_label_fontsize = w.IntSlider(value = 12, min = 8, max = 40, step = 1,
                                                description = 'fontsize:', layout = {'width': '28%'})
        self.yaxis_label_color = w.ColorPicker(concise = False, description = 'font color', 
                                               value = '#000000', layout = {'width': '28%'})
        self.yaxis_scaling_mode = w.RadioButtons(description = 'Please select whether you want to use \
                                                 automatic or manual scaling of the yaxis:',
                                                 options = [('Use automatic scaling', 'auto'), 
                                                            ('Use manual scaling', 'manual')],
                                                 value = 'auto', layout = {'width': '90%', 'height': '75px'}, 
                                                 style = {'description_width': 'initial'})
        data_column_values = self.gui.session.database.stats_results['df_infos']['data_column_values']
        if data_column_values.min() < 0:
            yaxis_lower_lim_value = round(data_column_values.min() + data_column_values.min()*0.1, 2)
        else:
            yaxis_lower_lim_value = round(data_column_values.min() - data_column_values.min()*0.1, 2)
        if data_column_values.max() < 0:
            yaxis_upper_lim_value = round(data_column_values.max() - data_column_values.max()*0.1, 2)
        else:
            yaxis_upper_lim_value = round(data_column_values.max() + data_column_values.max()*0.1, 2)
        self.yaxis_lower_lim = w.FloatText(value = yaxis_lower_lim_value, description = 'lower limit:', 
                                           style = {'description_width': 'initial'}, layout = {'width': '45%'})
        self.yaxis_upper_lim = w.FloatText(value = yaxis_upper_lim_value, description = 'upper limit:', 
                                           style = {'description_width': 'initial'}, 
                                           layout = {'width': '45%'})
        row_0 = w.HBox([self.yaxis_label_text, self.yaxis_label_fontsize, self.yaxis_label_color])
        row_1 = self.yaxis_scaling_mode
        row_2 = w.HBox([self.yaxis_lower_lim, self.yaxis_upper_lim])
        return w.VBox([row_0, row_1, row_2])
    
    
    def initialize_common_axis_features_vbox(self):
        
        self.axes_linewidth = w.BoundedFloatText(value = 1, min = 0, max = 40, description = 'Axes linewidth',
                                                 style = {'description_width': 'initial'}, layout = {'width': '30%'})
        self.axes_color = w.ColorPicker(concise = False, description = 'Axes and tick label color',
                                        value = '#000000', style = {'description_width': 'initial'}, 
                                        layout={'width': '30%'})
        self.axes_tick_size = w.BoundedFloatText(value = 10, min = 1, max = 40, description = 'Tick label size',
                                                style = {'description_width': 'initial'}, layout = {'width': '30%'})
        row_0 = w.HBox([self.axes_linewidth, self.axes_color, self.axes_tick_size])
        return w.VBox([row_0])
        
        

    def initialize_additional_configs_accordion_tab(self) -> VBox:
        self.fig_height = w.FloatSlider(value=28, min=1, max=50, description='Figure width:', 
                                       style={'description_width': 'inital'})
        self.fig_width = w.FloatSlider(value=16, min=1, max=50, description='Figure height:', 
                                        style={'description_width': 'inital'})
        self._preset_color_palette = w.Dropdown(options = ['colorblind', 'Spectral', 'viridis', 'rocket', 'cubehelix'],
                                                value = 'viridis',
                                                description = 'Select a color palette',
                                                layout = {'width': '35%'},
                                                style = {'description_width': 'initial'})
        color_pickers = []
        for group_id in self.gui.session.database.stats_results['df_infos']['all_group_ids']:
            color_pickers.append(w.ColorPicker(concise = False, description = group_id, style = {'description_width': 'initial'}))
        self._custom_color_palette = w.VBox(color_pickers)
        self.color_palette = w.RadioButtons(description = 'Please select a color code option and chose from the respective options below:',
                                            options = [('Use a pre-defined palette', 'preset'), 
                                                       ('Define colors individually', 'custom')],
                                            value = 'preset', layout = {'width': '80%', 'height': '75px'}, 
                                            style = {'description_width': 'initial'})
        self.show_legend = w.Checkbox(value=True, description='Show legend (if applicable):', 
                                      style={'description_width': 'initial'})
        self.marker_size = w.FloatText(value=5,description='marker size (if applicable):', 
                                       style={'description_width': 'initial'})
        
        row_0 = w.HBox([self.fig_height, self.fig_width])
        row_1 = w.VBox([self.color_palette, 
                        w.HBox([self._preset_color_palette, self._custom_color_palette])])
        row_2 = w.HBox([self.show_legend, self.marker_size])
        return w.VBox([row_0, row_1, row_2])
        
                                             
    def export_current_settings(self, b) -> None:
        pass
    
    
    def extract_and_set_uploaded_configs(self, b) -> None:
        pass
        
        
    def update_configs(self) -> Configs:
        configs = self.gui.session.database.configs
        setattr(configs, 'plot_type', self.gui.plot_tab.select_plot_type.value)
        for attr in self.__dict__.keys():
            if hasattr(configs, attr):
                if attr == 'color_palette':
                    color_palette = self.handle_color_palette_exception()
                    setattr(configs, 'color_palette', color_palette)
                else:
                    widget = getattr(self, attr)
                    value = widget.value
                    setattr(configs, attr, value)
            elif attr == 'l_xlabel_order':
                l_xlabel_order = self.handle_l_xlabel_order_exception()
                setattr(configs, 'l_xlabel_order', l_xlabel_order)
        return configs
        
    
    
    def handle_color_palette_exception(self) -> Union[str, Dict]:
        if self.color_palette.value == 'preset':
            color_palette = self._preset_color_palette.value
        else:
            color_palette = dict()
            for group_id, color_picker in zip(self.gui.session.database.stats_results['df_infos']['all_group_ids'], 
                                              self._custom_color_palette.children):
                color_palette[group_id] = color_picker.value
        return color_palette
        
    
    def handle_l_xlabel_order_exception(self) -> List[str]:
        l_xlabel_order = []
        l_xlabel_string = self.l_xlabel_order.value
        while ', ' in l_xlabel_string:
            l_xlabel_order.append(l_xlabel_string[:l_xlabel_string.index(', ')])
            l_xlabel_string = l_xlabel_string[l_xlabel_string.index(', ')+2:]
        l_xlabel_order.append(l_xlabel_string)
        return l_xlabel_order

            

        

In [ ]:
launch_gui()

In [ ]:
w.Accordion([w.HBox([w.Button()]), w.Accordion([w.Button(), w.Button()])])

In [ ]:
class Hund:
    def __init__(self):
        self.age = 5

In [ ]:
a = Hund()

for attr in a.__dict__.keys():
    if attr == 'age':
        print(attr)

age


In [ ]:
w.Dropdown(description = 'Please select a plot type: ',
           options = ['1', '2'], 
           layout = {'width': '50%'},
           style = {'description_width': 'initial'})

Dropdown(description='Please select a plot type: ', layout=Layout(width='50%'), options=('1', '2'), style=Desc…

In [ ]:
hund = ''
f'1- {hund}2'

'1- 2'

In [ ]:
GUI().initial_selection_widget

In [ ]:





class GUI:
    
    def __init__(self):
        self.please_specify_root_dir = w.Label(value='Please select a file (.xlsx or .csv) you would like to upload:')
        if os.path.isdir('/home/pi/Desktop/'):
            default_path = '/home/pi/Desktop/'
        else:
            default_path = '/mnt/c/Users/dsege/OneDrive/Desktop/'
        
        self.file_chooser = FileChooser(default_path)
        self.file_chooser.show_only_dirs = True
        
        self.confirm_root_dir_button = w.Button(description='confirm path')
        self.confirm_root_dir_button.on_click(self.build_remaining_gui)
        
        self.widget = w.VBox([self.please_specify_root_dir,
                              w.HBox([self.file_chooser, self.confirm_root_dir_button])])


    def build_remaining_gui(self, b):
        
        self.api = API(root_dir = self.file_chooser.selected_path + '/')
        
        self.recordings_tab = RecordingsTab(api = self.api)
        self.analysis_tab = AnalysisTab(api = self.api)
        self.plotting_tab = PlottingTab(api = self.api)
        
        self.navigator = Navigator(parent_widget = self.widget,
                                   parent_recordings_tab = self.recordings_tab,
                                   parent_analysis_tab = self.analysis_tab, 
                                   parent_plotting_tab = self.plotting_tab)
        
        self.widget.children = (self.navigator.widget, )
        
       
    
class Navigator:
    
    def __init__(self, parent_widget, parent_recordings_tab, parent_analysis_tab, parent_plotting_tab):
        self.parent_widget = parent_widget
        self.parent_recordings_tab = parent_recordings_tab
        self.parent_analysis_tab = parent_analysis_tab
        self.parent_plotting_tab = parent_plotting_tab
        
        self.record_button = w.Button(description='1. Recordings', style={'button_color': 'lightgray'})
        self.analyze_button = w.Button(description='2. Analysis', style={'button_color': 'lightgray'})
        self.plot_button = w.Button(description='3. Plotting', style={'button_color': 'lightgray'})
        self.widget = w.HBox([self.record_button, self.analyze_button, self.plot_button])
        
        self.record_button.on_click(self.record_button_clicked)
        self.analyze_button.on_click(self.analyze_button_clicked)
        self.plot_button.on_click(self.plot_button_clicked)
        
    def record_button_clicked(self, b):
        self.record_button.style.button_color = 'lightblue'
        self.analyze_button.style.button_color = 'lightgray'
        self.plot_button.style.button_color = 'lightgray'
        self.switch_interface(b, new_widget=self.parent_recordings_tab.widget)
    
    
    def analyze_button_clicked(self, b):
        self.record_button.style.button_color = 'lightgray'
        self.analyze_button.style.button_color = 'lightblue'
        self.plot_button.style.button_color = 'lightgray'
        self.switch_interface(b, new_widget=self.parent_analysis_tab.widget)
        
        
    def plot_button_clicked(self, b):
        self.record_button.style.button_color = 'lightgray'
        self.analyze_button.style.button_color = 'lightgray'
        self.plot_button.style.button_color = 'lightblue'
        self.switch_interface(b, new_widget=self.parent_plotting_tab.widget)
    
    
    def switch_interface(self, b, new_widget):
        self.parent_widget.children = w.VBox([self.widget, new_widget]).children
        


class RecordingsTab:
    
    def __init__(self, api):
        self.api = api
        
        self.please_choose = w.Label(value='Please provide the following details for your experiment:')
        
        self.select_group_id = w.Dropdown(description='Group ID:',
                                          options=['wt', 'tg'],
                                          value='wt',
                                          layout={'width': '30%'},
                                          style={'description_width': 'initial'})
        self.select_stimulus_indicator = w.Dropdown(description='Stimulus indicator:', 
                                                   options=['pre', 'post'],
                                                   value='pre',
                                                   layout={'width': '30%'},
                                                   style={'description_width': 'initial'})
        self.select_vial_id = w.Dropdown(description='Vial ID:',
                                        options=[1, 2, 3, 4],
                                        value=1,
                                        layout={'width': '30%'},
                                        style={'description_width': 'initial'})
        
        self.confirm_selection_button = w.Button(description='confirm selection')
        
        self.trigger_recording_button = w.Button(description='trigger recording',
                                                layout={'visibility': 'hidden'})
        
        self.widget = w.VBox([self.please_choose,
                              w.HBox([self.select_group_id,
                                      self.select_vial_id,
                                      self.select_stimulus_indicator]),
                              w.HBox([self.confirm_selection_button,
                                      self.trigger_recording_button])])
        
        self.confirm_selection_button.on_click(self.confirm_selection_button_clicked)
        self.trigger_recording_button.on_click(self.trigger_recording_button_clicked)
        
    def confirm_selection_button_clicked(self, b):
        self.trigger_recording_button.layout.visibility = 'visible'
        self.confirm_selection_button.layout.visibility = 'hidden'
        
    def trigger_recording_button_clicked(self, b):
        self.trigger_recording_button.layout.visibility = 'hidden'
        self.confirm_selection_button.layout.visibility = 'visible'
        
        group_id = self.select_group_id.value
        vial_id = self.select_vial_id.value
        stimulus_indicator = self.select_stimulus_indicator.value
        
        self.api.record_experiment(group_id = group_id, vial_id = vial_id, stimulus_indicator = stimulus_indicator)
